In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier as XGB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

Duplicate key in file WindowsPath('C:/Users/chikachika/anaconda3/Lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 263 ('font.family:  IPAexGothic')


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
#Cabinを分割且つ、欠損値補完
train["deck"] = train["Cabin"].apply(lambda x :x.split("/")[0] if isinstance(x, str) else "unknown")
train["num"] = train["Cabin"].apply(lambda x :x.split("/")[1] if isinstance(x, str) else "unknown")
train["side"] = train["Cabin"].apply(lambda x :x.split("/")[2] if isinstance(x, str) else "unknown")

test["deck"] = test["Cabin"].apply(lambda x :x.split("/")[0] if isinstance(x, str) else "unknown")
test["num"] = test["Cabin"].apply(lambda x :x.split("/")[1] if isinstance(x, str) else "unknown")
test["side"] = test["Cabin"].apply(lambda x :x.split("/")[2] if isinstance(x, str) else "unknown")

In [5]:
#Nameを姓と名に分割且つ、欠損値補完
train["first"] = train["Name"].apply(lambda x :x.split(" ")[0] if isinstance(x, str) else "unknown")
train["last"] = train["Name"].apply(lambda x :x.split(" ")[1] if isinstance(x, str) else "unknown")

test["first"] = test["Name"].apply(lambda x :x.split(" ")[0] if isinstance(x, str) else "unknown")
test["last"] = test["Name"].apply(lambda x :x.split(" ")[1] if isinstance(x, str) else "unknown")

In [6]:
#欠損値補完
train["Destination"].fillna("missingPL", inplace=True)
test["Destination"].fillna("missingPL", inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\2812334904.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Destination"].fillna("missingPL", inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\2812334904.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [7]:
train["VIP"].value_counts()

VIP
False    8291
True      199
Name: count, dtype: int64

In [8]:
#Age欠損値を平均値で補完
train["Age"].fillna(train["Age"].mean(),inplace=True)
test["Age"].fillna(test["Age"].mean(),inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\628498002.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["Age"].fillna(train["Age"].mean(),inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\628498002.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [9]:
train[train["VRDeck"] >= 1]["VIP"].value_counts()

VIP
False    2829
True      118
Name: count, dtype: int64

In [10]:
#HomePlanet欠損値補完
train["HomePlanet"].fillna("missing",inplace=True)
test["HomePlanet"].fillna("missing",inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\4141982052.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["HomePlanet"].fillna("missing",inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\4141982052.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [11]:
train["CryoSleep"].value_counts()

CryoSleep
False    5439
True     3037
Name: count, dtype: int64

In [12]:
#欠損値補完
train["CryoSleep"].fillna("None",inplace=True)
test["CryoSleep"].fillna("None",inplace=True)
train["VIP"].fillna("None",inplace=True)
test["VIP"].fillna("None",inplace=True)

train["RoomService"].fillna(0,inplace=True)
train["FoodCourt"].fillna(0,inplace=True)
train["ShoppingMall"].fillna(0,inplace=True)
train["Spa"].fillna(0,inplace=True)
train["VRDeck"].fillna(0,inplace=True)

test["RoomService"].fillna(0,inplace=True)
test["FoodCourt"].fillna(0,inplace=True)
test["ShoppingMall"].fillna(0,inplace=True)
test["Spa"].fillna(0,inplace=True)
test["VRDeck"].fillna(0,inplace=True)

C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\987248775.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["CryoSleep"].fillna("None",inplace=True)
C:\Users\chikachika\AppData\Local\Temp\ipykernel_17944\987248775.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   object 
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  deck          8693 non-null   object 
 15  num           8693 non-null   object 
 16  side          8693 non-null   object 
 17  first         8693 non-null   object 
 18  last          8693 non-null 

In [14]:
#PassengerIdの乗客IDを取り出す
train["PassengerId_gg"] = train["PassengerId"].apply(lambda x :x.split("_")[0] if isinstance(x, str) else "unknown")

# PassengerId_ggとlastの組み合わせで同じ人が何人いるか数える
train["Family_num"] = train.groupby(["PassengerId_gg", "last"])["PassengerId"].transform("count")

# Family_num：2人以上なら1、1人だけなら0
train["Family"] = train["Family_num"].apply(lambda x: 1 if x > 1 else 0)

# Family_num削除
train.drop(columns=["Family_num"], inplace=True)

#テストデータも同様に処理
test["PassengerId_gg"] = test["PassengerId"].apply(lambda x :x.split("_")[0] if isinstance(x, str) else "unknown")
test["Family_num"] = test.groupby(["PassengerId_gg", "last"])["PassengerId"].transform("count")
test["Family"] = test["Family_num"].apply(lambda x: 1 if x > 1 else 0)
test.drop(columns=["Family_num"], inplace=True)

In [15]:
#特徴量をdrop、目的変数設定
trainX = train.drop(["Cabin","Name","PassengerId","Transported","PassengerId_gg","first","last",],axis=1)
y = train["Transported"]
testX = test.drop(["Cabin","Name","PassengerId","PassengerId_gg","first","last",],axis=1)

In [16]:
trainX["num"].value_counts()

num
unknown    199
82          28
86          22
19          22
56          21
          ... 
1644         1
1515         1
1639         1
1277         1
1894         1
Name: count, Length: 1818, dtype: int64

In [17]:
#numカラム内補完した"unknown"を0に変換
trainX["num"] = trainX["num"].replace("unknown", 0).astype(int)
testX["num"] = testX["num"].replace("unknown", 0).astype(int)

In [18]:
#trainX,testXをダミー変数化
trainX = pd.get_dummies(trainX,dtype=int)
testX = pd.get_dummies(testX,dtype=int)

In [19]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        8693 non-null   float64
 1   RoomService                8693 non-null   float64
 2   FoodCourt                  8693 non-null   float64
 3   ShoppingMall               8693 non-null   float64
 4   Spa                        8693 non-null   float64
 5   VRDeck                     8693 non-null   float64
 6   num                        8693 non-null   int32  
 7   Family                     8693 non-null   int64  
 8   HomePlanet_Earth           8693 non-null   int32  
 9   HomePlanet_Europa          8693 non-null   int32  
 10  HomePlanet_Mars            8693 non-null   int32  
 11  HomePlanet_missing         8693 non-null   int32  
 12  CryoSleep_False            8693 non-null   int32  
 13  CryoSleep_True             8693 non-null   int32

In [20]:
#必要のないカラムを削除
trainX.drop(["HomePlanet_missing","CryoSleep_None","Destination_missingPL","VIP_None","deck_unknown","side_unknown"],axis=1,inplace=True)
testX.drop(["HomePlanet_missing","CryoSleep_None","Destination_missingPL","VIP_None","deck_unknown","side_unknown"],axis=1,inplace=True)

In [21]:
#目的変数を数値化
y = train["Transported"].astype(int)

In [44]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        8693 non-null   float64
 1   RoomService                8693 non-null   float64
 2   FoodCourt                  8693 non-null   float64
 3   ShoppingMall               8693 non-null   float64
 4   Spa                        8693 non-null   float64
 5   VRDeck                     8693 non-null   float64
 6   num                        8693 non-null   int32  
 7   Family                     8693 non-null   int64  
 8   HomePlanet_Earth           8693 non-null   int32  
 9   HomePlanet_Europa          8693 non-null   int32  
 10  HomePlanet_Mars            8693 non-null   int32  
 11  CryoSleep_False            8693 non-null   int32  
 12  CryoSleep_True             8693 non-null   int32  
 13  Destination_55 Cancri e    8693 non-null   int32

In [50]:
#XGBモデル
xgb = XGB(use_label_encoder=False, eval_metric="auc", random_state=42)

param_grid = {'max_depth': [3, 5],'learning_rate': [0.1, 0.3,0.5],'n_estimators': [50, 100,150]}

gcv_xgb = GridSearchCV(xgb, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [52]:
gcv_xgb.fit(trainX,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:55:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=No...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5], 'max_depth': [3, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [54]:
pred_xgb = gcv_xgb.predict(testX)

In [56]:
pred_xgb = pred_xgb.astype(bool)

In [58]:
trainX.corrwith(y)

Age                         -0.074249
RoomService                 -0.241124
FoodCourt                    0.045583
ShoppingMall                 0.009391
Spa                         -0.218545
VRDeck                      -0.204874
num                         -0.043832
Family                       0.100146
HomePlanet_Earth            -0.169019
HomePlanet_Europa            0.176916
HomePlanet_Mars              0.019544
CryoSleep_False             -0.451744
CryoSleep_True               0.460132
Destination_55 Cancri e      0.108722
Destination_PSO J318.5-22    0.000092
Destination_TRAPPIST-1e     -0.094700
VIP_False                    0.024602
VIP_True                    -0.037261
deck_A                      -0.002623
deck_B                       0.144733
deck_C                       0.108193
deck_D                      -0.034046
deck_E                      -0.097965
deck_F                      -0.087753
deck_G                       0.016269
deck_T                      -0.014568
side_P      

In [60]:
#LGBMモデル
model_LG = LGBMClassifier(use_label_encoder=False, eval_metric="auc", random_state=42)
gcv_LG = GridSearchCV(model_LG, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [62]:
gcv_LG.fit(trainX,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

GridSearchCV(cv=5,
             estimator=LGBMClassifier(eval_metric='auc', random_state=42,
                                      use_label_encoder=False),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5], 'max_depth': [3, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [64]:
gcv_LG.best_params_

{'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 50}

In [66]:
best_model = gcv_LG.best_estimator_

In [70]:
pred_LG = best_model.predict(testX).astype(bool)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric


In [74]:
#CatBoostモデル
model_cat = CatBoostClassifier(verbose=0)
gcv_cat = GridSearchCV(model_cat, param_grid,scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

In [76]:
gcv_cat.fit(trainX,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x000002A60E107DA0>,
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5], 'max_depth': [3, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [78]:
pred_cat = gcv_cat.predict(testX).astype(bool)

In [82]:
#スタッキング（XGB,LGBM,Cat）再グリッドサーチあり
estimators = [
    ('xgb', gcv_xgb),
    ('lgb', gcv_LG),
    ('cat', gcv_cat)
]

stacked_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)


In [84]:
stacked_model.fit(trainX,y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:01:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:02:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


...

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:02:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:02:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000894 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1649
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

StackingClassifier(estimators=[('xgb',
                                GridSearchCV(cv=5,
                                             estimator=XGBClassifier(base_score=None,
                                                                     booster=None,
                                                                     callbacks=None,
                                                                     colsample_bylevel=None,
                                                                     colsample_bynode=None,
                                                                     colsample_bytree=None,
                                                                     device=None,
                                                                     early_stopping_rounds=None,
                                                                     enable_categorical=False,
                                                                     eval_metric='auc',
                                                                     feature_types=None,
                                                                     feature_weights=None,
                                                                     gamma=None,
                                                                     grow_policy=None,
                                                                     importanc...
                                                                           0.3,
                                                                           0.5],
                                                         'max_depth': [3, 5],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          150]},
                                             scoring='accuracy', verbose=1)),
                               ('cat',
                                GridSearchCV(cv=5,
                                             estimator=<catboost.core.CatBoostClassifier object at 0x000002A60E107DA0>,
                                             n_jobs=-1,
                                             param_grid={'learning_rate': [0.1,
                                                                           0.3,
                                                                           0.5],
                                                         'max_depth': [3, 5],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          150]},
                                             scoring='accuracy', verbose=1))],
                   final_estimator=LogisticRegression())

In [86]:
pred_stacked = stacked_model.predict(testX).astype(bool)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric


In [88]:
#スタッキング（XGB,LGBM,Cat）ベストモデルのみ
estimators_best = [
    ('xgb', gcv_xgb.best_estimator_),
    ('lgb', gcv_LG.best_estimator_),
    ('cat', gcv_cat.best_estimator_)
]

stacked_model_best = StackingClassifier(
    estimators=estimators_best,
    final_estimator=LogisticRegression()
)

In [90]:
stacked_model_best.fit(trainX,y)

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 4378, number of negative: 4315
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1650
[LightGBM] [Info] Number of data points in the train set: 8693, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503624 -> initscore=0.014495
[LightGBM] [Info] Start training from score 0.014495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\chikachika\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:09:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not use

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 3502, number of negative: 3452
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1649
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503595 -> initscore=0.014380
[LightGBM] [Info] Start training from score 0.014380
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='auc',
                                              feature_types=None,
                                              feature_weights=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_cons...
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None, ...)),
                               ('lgb',
                                LGBMClassifier(eval_metric='auc',
                                               learning_rate=0.3, max_depth=3,
                                               n_estimators=50, random_state=42,
                                               use_label_encoder=False)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x000002A60DDE15E0>)],
                   final_estimator=LogisticRegression())

In [92]:
pred_stacked_best = stacked_model_best.predict(testX).astype(bool)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
